In [3]:
import numpy as np
import sys

# Optimal Paths and Dynamic Programming

Let's play a game. You start off at the upper left corner of a large, $N\times N$ checkerboard. Each square has some number of gold coins on it, and you are only allowed to move to squares adjacent to you own (no diagonals). You're goal is to collect as many gold coins as possible as you make your way to the exit located at the bottom right corner.

But there's a catch...

Every second that passes brings you closer to a ghastly, untimely demise. As a result, you can't backtrack under any circumstances. Lucky for you, you have a friend that has a nice view of the board, and can communicate in your ear. How can your friend guide you on the path that yields the most bounty without your own downfall?

Your friend would like to check *every* allowed path before giving you advice, but you don't have time! Even as you enter the game you can here the fell footsteps of the fell Balgrog making his way toward you. What oh what are you to do! 

# Dynamic Programming
Enter stage-left the hero, the deus ex machina: dynamic programming! This is essentially a brute force method. Just like your friend wanted to do, let's try every path! But let's be clever about it so we stay a step ahead of our impending doom.

Dynamic programming is useful when a problem can be broken down in to subproblems. Let us consider our game. We are starting at the point $\mathcal{O}\equiv (0,0)$ and want to find the optimal path to $\mathcal{N}\equiv (N-1, N-1)$. We denote this $\mathcal{O}\rightarrow \mathcal{N}$. Now suppose the shortest path includes the square $\mathcal{U}_n\equiv (i_n,j_n)$ as some intermediary point. Then we also know that $\mathcal{O}\rightarrow \mathcal{U}_n$ is an optimal path.

Our goal is then the following. Suppose we have already made it to the point $\mathcal{U}_n$. Then our approach is to try every path from $\mathcal{U}_n\rightarrow \mathcal{N}$ and take that which gives the highest score.

We can describe this in a more concrete way. Suppose the function $F(n,m)$ returns the optimal path to the square indexed by $(n,m)$ and $\mathcal{G}(n,m)$ is the number of coins on square $(n,m)$. We know that this path must pass through either $(n-1, m)$ or $(n, m-1)$. So, if $F(n,m)$ returns the optimal path. So, we can write $F(n,m)$ recursively:
$$
  F(n,m) = \mathcal{G}(n,m) + \mathrm{max}\lbrace F(n-1, m), F(n, m-1)\rbrace.
$$


In the following cell we implement this recursion.

In [230]:
def bestPath(coins, n, m):
    """Returns the maximum number of gold coins attainable while traversing the grid,
    assuming no backtracking is allowed. We utilize a dynamic programming algorith:
    recursion + memoization.
    
    It is assumed user will initialize global memo dict before calling function."""
    memo_key = str(n)+','+str(m)
    if memo_key in memo_pad:
        return memo_pad[memo_key]
    else:
        if (n < 0 or m < 0):
            # This means we are off the grid. Return negative coins
            # so that it will always be less than any square on the grid.
            memo_pad[memo_key] = -sys.maxsize
            return memo_pad[memo_key]
        elif (n==0 and m==0):
            # This is our base case. Return the
            # number of coins on first square
            memo_pad[memo_key] = coins[n, m]
            return memo_pad[memo_key]
        else:
            # Recursively call bestPath
            memo_pad[memo_key] = coins[n, m] + max([bestPath(coins, n-1, m), bestPath(coins, n, m-1)])
            return memo_pad[memo_key]

<blockquote> Note in the above cell we included a memo pad </blockquote>

Next we randomly generate a square grid filled with random integers. Each entry of the array represents the number of gold coins on that square.

In [231]:
N = 100
coins = np.random.randint(0, 10, N**2).reshape(N, N)
memo_pad = {}
max_score = bestPath(coins, N-1, N-1)
print('Congratulations! You survived the trial and collected {} gold coins in the process!'.format(max_score))

Congratulations! You survived the trial and collected 1351 gold coins in the process!


And that's it! Aside from the Fibonacci numbers, it is probably the simplest example of DP. There are great lectures on this method by Eric Demaine hosted on YouTube. If you are interested in this method, you should check them out!

# Finding the actual path
The above exercise was fun, but if you think carefully about it, it doesn't help you in your plight. All we did was determine the number of gold points you will get if you take the optimal path, but what you really need to know is what the optimal path is!

We can get this information (i.e. right, down, right, right, down,...) by including a parent pointer. Namely, when we calculate the maximum of incoming paths, we also keep track of *which* step that maximum corresponds to. The object we use to do this is called the *parent pointer*. There is a catch to the way we are implementing it, as we will see.

In [232]:
def bestestPath(coins, n, m):
    """Returns the maximum number of gold coins attainable while traversing the grid,
    assuming no backtracking is allowed. We utilize a dynamic programming algorith:
    recursion + memoization. We use a parent pointer to keep track of the best actual
    path.
    
    It is assumed that there exists a global memo and parent dictionary."""
    memo_key = str(n)+','+str(m)
    if memo_key in memo_pad:
        return memo_pad[memo_key]
    else:
        if (n < 0 or m < 0):
            # This means we are off the grid. Return negative coins
            # so that it will always be less than any square on the grid.
            memo_pad[memo_key] = -sys.maxsize
            return memo_pad[memo_key]
        elif (n==0 and m==0):
            # This is our base case. Return the
            # number of coins on first square
            memo_pad[memo_key] = coins[n, m]
            return memo_pad[memo_key]
        else:
            # Recursively call bestestPath
            memo_pad[memo_key] = coins[n, m] + max([bestestPath(coins, n-1, m), bestestPath(coins, n, m-1)])
            arg_max = np.argmax([bestestPath(coins, n-1, m), bestestPath(coins, n, m-1)])
            #print(memo_key, arg_min)
            parent[memo_key] = arg_max
            return memo_pad[memo_key]

In [233]:
N = 3
coins = np.random.randint(0, 10, N**2).reshape(N, N)
coins[0,0] = 0
print('Your grid:\n' + str(coins))
memo_pad = {}
parent = {}
max_score = bestestPath(coins, N-1, N-1)
print('Congratulations! You survived the trial and collected {} gold coins in the process!'.format(max_score))

Your grid:
[[0 8 6]
 [6 2 1]
 [1 0 4]]
Congratulations! You survived the trial and collected 19 gold coins in the process!


We have also printed the grid for our inspection. We can see that the best path is to move down two spaces (collect

We have to untangle our parent pointer a little bit. This is not inherent to parent pointers, but is simply a consequence of my limited fluency with dynamic programming. 

To see what the issue is, let's pring the parent

In [234]:
parent

{'0,1': 1,
 '0,2': 1,
 '1,0': 0,
 '1,1': 0,
 '1,2': 0,
 '2,0': 0,
 '2,1': 0,
 '2,2': 0}

Notice that we have a value for every square in the grid, yet we know the optimal path *cannot* visit every square (it will always be $N-1$ steps). The parent pointer tells us that the optimal path for element $(n,m)$ comes in from the top (0) or the left (1). To interpret the parent, we define a list  `path` and populate it by starting at the last square in the grid $N-1, N-1$ and peform the following:
* if the value of parent is 0, then we append an 'r' to the front of path and repeat for the cell to the left.
* if the value of parent is 1, then we append a 'd' to the front of path and repeat for the cell above.

We write a function that performs this conversion, then check the result against our grid:

In [235]:
def get_path(parent, N):
    path = ''
    n,m = (N-1, N-1)
    while (n,m) != (0,0):
        parent_key = str(n)+','+str(m)
        step = parent[parent_key]
        if step==1:
            path = 'right, '+path
            m -= 1
        if step==0:
            path = 'down, '+path
            n -= 1
    return path
            
path = get_path(parent, N)
print(coins)
print(path)

[[0 8 6]
 [6 2 1]
 [1 0 4]]
right, right, down, down, 


We can verify that this indeed corresponds to the optimal path!

# Extensions and Generalizations
Some future extensions I would like to implement
1. We could enforce the time constraint in a different manner. In the above, the time constraint was accounted for by explicitly forbiding backtracking. This implicitly sets an upper limit on the number of steps taken. We could instead simply set a maximum number of steps and allow backtracking if it still lets one complete the course in the given number of steps. Here are a few thoughts about this possibility:
   * If implemented naively, this will lead to closed loops and thus an infinite recursive algorithm.
   * It *might* be possible to avoid the infinite loops by removing gold coins as we come across them. It's not clear how to implement this in practice. Furthermore, I think it would make memoization impossible.